In [1]:
import jVMC
import jax.numpy as jnp 
import jax.random as jrnd
import numpy as np 
import jVMC.nets.bosons


In [2]:
N = 10
L = 10
ldim = 8
homFock = jnp.ones((1,1,L),dtype=int)

emb_RWKV = 8
depth_RWKV = 2
net_RWKV = jVMC.nets.bosons.RWKV(L,N,LocalHilDim=ldim,num_layers=depth_RWKV,embedding_size=emb_RWKV)


psi_RWKV = jVMC.vqs.NQS(net_RWKV)

#dummies for testing
net_FFN = jVMC.nets.ffn.FFN()
psi_FFN = jVMC.vqs.NQS(net_FFN)

psi_RWKV(homFock)


Array([[-7.70078432]], dtype=float64)

In [4]:
class MCSampler_gumbel(jVMC.sampler.MCSampler):
    
    def __init__(self,*args,**kwargs):
        super().__init__(*args, **kwargs)
        try: 
            callable(self.net.net.forward_with_state)
        except:
            raise Exception("neural network has no autoregressive stepwise sampling subroutine 'forward_with_state'")
    def _scanning_fn_gumble(self, carry, key):
        
        logits_i, next_states = self.net(carry[0],block_states = carry[1], output_state=True)
        #carry[0] = [x_<i,....]
        #carry[1] = [logits_<i,....]
        #carry[2] = [logits_gumble<i,....]
        
        ### commented out the particle conservation
        ### maybe just add the self -> self.net
        ##############################################
        # mask logits
        # number of particles that mus assigned
        ##must_give = nn.relu(carry[2]-self.max_particles[key[1]])
        # number of particles that can be assigned
        ##can_give = jnp.minimum(carry[2], self.LocalHilDim-1)
        # compute mask
        ##mask = (self.must_mask[must_give] + 
        ##            self.can_mask[can_give.astype(int)]) 
        ##############################################
        states = jnp.expand_dims(
        logits = (carry[1] + logits_i[:,None]).ravel()
        
        logits_gumble = logits + jrnd.gumbel(key[0],logits.shape)##- mask ** jnp.inf
        ##choice = jrnd.categorical(key[0], logits.ravel().real) # abide by the indexing convention
        ##############################################
        ##s_cumsum = carry[2] - choice
        return (logits, logits_gumble,next_states, s_cumsum), choice
        #return (jnp.expand_dims(choice,0), next_states, s_cumsum), choice
    def _get_samples_gen_gumbel():
        numSamples = mpi.distribute_sampling(numSamples, localDevices=global_defs.device_count(), numChainsPerDevice=multipleOf)

        tmpKeys = random.split(self.key[0], 3 * global_defs.device_count())
        self.key = tmpKeys[:global_defs.device_count()]
        tmpKey = tmpKeys[global_defs.device_count():2 * global_defs.device_count()]
        tmpKey2 = tmpKeys[2 * global_defs.device_count():]

        #### rewrite explicit scanning in here
        samples = self.net.sample(numSamples, tmpKey, parameters=params)
        ####
        if not str(numSamples) in self._randomize_samples_jitd:
            self._randomize_samples_jitd[str(numSamples)] = global_defs.pmap_for_my_devices(self._randomize_samples, static_broadcasted_argnums=(), in_axes=(0, 0, None))

        if not self.orbit is None:
            return self._randomize_samples_jitd[str(numSamples)](samples, tmpKey2, self.orbit)
        
        return samples
    def sample(self, parameters=None, numSamples=None, multipleOf=1):
        """Generate random samples from wave function.

        If supported by ``net``, direct sampling is peformed. Otherwise, MCMC is run \
        to generate the desired number of samples. For direct sampling the real part \
        of ``net`` needs to provide a ``sample()`` member function that generates \
        samples from :math:`p_{\\mu}(s)`.

        Sampling is automatically distributed accross MPI processes and available \
        devices. In that case the number of samples returned might exceed ``numSamples``.

        Arguments:
            * ``parameters``: Network parameters to use for sampling.
            * ``numSamples``: Number of samples to generate. When running multiple processes \
            or on multiple devices per process, the number of samples returned is \
            ``numSamples`` or more. If ``None``, the default number of samples is returned \
            (see ``set_number_of_samples()`` member function).
            * ``multipleOf``: This argument allows to choose the number of samples returned to \
            be the smallest multiple of ``multipleOf`` larger than ``numSamples``. This feature \
            is useful to distribute a total number of samples across multiple processors in such \
            a way that the number of samples per processor is identical for each processor.

        Returns:
            A sample of computational basis configurations drawn from :math:`p_{\\mu}(s)`.
        """

        if numSamples is None:
            numSamples = self.numSamples

        
        if parameters is not None:
            tmpP = self.net.params
            self.net.set_parameters(parameters)

        
        configs = self._get_samples_gen_gumbel(self.net.parameters, numSamples, multipleOf)
        coeffs = self.net(configs)
        if parameters is not None:
            self.net.params = tmpP
        return configs, coeffs, jnp.ones(configs.shape[:2]) / jnp.prod(jnp.asarray(configs.shape[:2]))


In [5]:
seed = 1
key = jrnd.PRNGKey(seed)
sampler = jVMC.sampler.MCSampler(psi_RWKV,(L,),key)
sampler_gumbel = MCSampler_gumbel(psi_RWKV,(L,),key)
sampler_gumbel = MCSampler_gumbel(psi_FFN,(L,),key,updateProposer=jVMC.sampler.propose_spin_flip)

Exception: neural network has no autoregressive stepwise sampling subroutine 'forward_with_state'

In [ ]:
sampler_gumbel.sampleShape[0]